# Interactive Flight Route Map (Plotly)

This notebook provides an interactive world map of your flights using Plotly. It's a modern alternative to Basemap and works well inside JupyterLab/Notebook.

Usage: run all cells, paste or upload routes (one per line, e.g. `HKG,HGH`), then click **Draw**. The map is interactive (pan/zoom, hover).

In [1]:
import pandas as pd
from collections import Counter
import plotly.graph_objects as go
from pyproj import Geod


In [2]:
csv_path = "data/my_flight_log.csv"

full_routes = []
try:
    df = pd.read_csv(csv_path)
    for _, row in df.iterrows():
        o = str(row["origin"]).strip().upper()
        d = str(row["destination"]).strip().upper()
        if o and d and o != "NAN" and d != "NAN":
            full_routes.append((o, d))
except Exception as e:
    print(f"[WARN] Failed to read CSV: {e}")


In [3]:
# STEP 2: Define coordinates for all involved airports
airport_coords = {
    "HGH": (120.4333, 30.2295), "HKG": (113.9185, 22.3080), "BNE": (153.1094, -27.3842),
    "SYD": (151.1772, -33.9461), "PVG": (121.8052, 31.1443), "FRA": (8.5706, 50.0333),
    "TXL": (13.2877, 52.5597), "SIN": (103.9894, 1.3644), "PEK": (116.5975, 40.0801),
    "ALA": (76.8844, 43.3521), "NQZ": (71.4669, 51.0222), "SFO": (-122.375, 37.6189),
    "LAX": (-118.4085, 33.9416), "RDU": (-78.7875, 35.8776), "EWR": (-74.1745, 40.6895),
    "MSN": (-89.3375, 43.1399), "MKE": (-87.8966, 42.9481), "STL": (-90.3700, 38.7487),
    "SEA": (-122.3093, 47.4502), "OAK": (-122.221, 37.7126), "SAN": (-117.1897, 32.7336),
    "SJC": (-121.9290, 37.3627), "PDX": (-122.5975, 45.5898), "BOI": (-116.2228, 43.5644),
    "BOS": (-71.0052, 42.3656), "LAS": (-115.1523, 36.0833), "TPE": (121.2325, 25.0777),
    "DFW": (-97.0403, 32.8998), "JAN": (-90.0759, 32.3112), "YVR": (-123.1830, 49.1951),
    "TAO": (120.3744, 36.2661), "WNZ": (120.8530, 27.9122), "CAN": (113.2988, 23.3924),
    "SZX": (113.8107, 22.6393), "NRT": (140.3929, 35.7668), "BWN": (114.9283, 4.9442),
    "MFM": (113.5925, 22.1496), "ABQ": (-106.6092, 35.0496), "MSP": (-93.2218, 44.8848),
    "AUS": (-97.6699, 30.1945), "IAH": (-95.3414, 29.9844), "HOU": (-95.2789, 29.6454),
    "MIA": (-80.2906, 25.7959), "JFK": (-73.7781, 40.6413), "LHR": (-0.4543, 51.4700),
    "MCO": (-81.3081, 28.4312), "BNA": (-86.6782, 36.1245), "ONT": (-117.6000, 34.0556),
    "GSP": (-82.2206, 34.8954), "CHS": (-80.0405, 32.8986), "CLT": (-80.9431, 35.2140),
    "ATL": (-84.4277, 33.6407),
}

In [4]:
def normalize_route(o, d):
    return tuple(sorted([o, d]))

route_counts = Counter(
    normalize_route(o, d) for o, d in full_routes
)


In [5]:
geod = Geod(ellps="WGS84")

def great_circle(lon1, lat1, lon2, lat2, npts=80):
    """
    返回一条大圆路径（lon list, lat list）
    """
    pts = geod.npts(lon1, lat1, lon2, lat2, npts)
    lons = [lon1] + [p[0] for p in pts] + [lon2]
    lats = [lat1] + [p[1] for p in pts] + [lat2]
    return lons, lats


In [6]:
fig = go.Figure()

for (origin, dest), count in route_counts.items():
    if origin not in airport_coords or dest not in airport_coords:
        continue

    lon1, lat1 = airport_coords[origin]
    lon2, lat2 = airport_coords[dest]

    lons, lats = great_circle(lon1, lat1, lon2, lat2)

    fig.add_trace(
        go.Scattergeo(
            lon=lons,
            lat=lats,
            mode="lines",
            line=dict(
                width=1 + 0.6 * (count - 1),
                color="darkred",
            ),
            opacity=0.5 + 0.1 * min(count, 5),
            hoverinfo="text",
            text=f"{origin} → {dest} (×{count})",
        )
    )


In [7]:
fig.update_layout(
    title="Flight Routes (Great Circle, Pacific-Centered)",
    showlegend=False,
    geo=dict(
        lonaxis=dict(range=[-180, 180]),   # ⭐ 强制 360°
        lataxis=dict(range=[-60, 75]),     # 控制上下范围
        projection_type="mercator",
        projection_rotation=dict(lon=160),  # ⭐ Pacific-centered
        showcountries=True,
        showcoastlines=True,
        showland=True,
        landcolor="rgb(230,230,230)",
        showocean=True,
        oceancolor="rgb(210,230,255)",
    ),
    height=800,
)
